In [1]:
continuous_series_names = [
                           'uncorrelated_gaussian_centered_sigma_0.1_KDE',
                           'uncorrelated_gaussian_centered_sigma_0.3_KDE',                           
                           'uncorrelated_gaussian_centered_sigma_0.5_KDE',
                           'uncorrelated_gaussian_centered_sigma_0.8_KDE',
                           'uncorrelated_gaussian_centered_sigma_0.1_histogram',
                           'uncorrelated_gaussian_centered_sigma_0.3_histogram',
                           'uncorrelated_gaussian_centered_sigma_0.5_histogram',
                           'uncorrelated_gaussian_centered_sigma_0.8_histogram',

                           ]
markov_chain_names = ['markov_chain']

import numpy as np

### Set up directory
import sys
import os
from pathlib import Path
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from tqdm import tqdm
import pickle
import torch
import matplotlib.pyplot as plt

# Check if directory exists, if not create it
save_path = Path(parent_dir) / 'processed_series'
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
# Define the directory where the generated series are stored
generated_series_dir = Path(parent_dir) / 'generated_series'

In [2]:
# Check if CUDA is available and set the device accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available.")
else:
    device = torch.device("cpu")
    print("CUDA is not available.")


CUDA is available.


In [3]:
# Initialize dictionaries to store the data for continuous series and Markov chains
continuous_series_task = {}
markov_chain_task = {}

# Loop through each file in the directory
for file in generated_series_dir.iterdir():
    # Check if a series is already processed
    if not (save_path / file.name).exists():\
        # Extract the series name from the file name
        series_name = file.stem.rsplit('_', 1)[0]
        # If the series is a continuous series, load the data into the continuous_series_data dictionary
        if series_name in continuous_series_names:
            continuous_series_task[file.name] = pickle.load(file.open('rb'))
        # If the series is a Markov chain, load the data into the markov_chain_data dictionary
        elif series_name in markov_chain_names:
            markov_chain_task[file.name] = pickle.load(file.open('rb'))
        # If the series name is not recognized, raise an exception
        # else:
        #     raise Exception(f"Unrecognized series name: {series_name}")
        


In [4]:
print(continuous_series_task.keys())
print(markov_chain_task.keys())

dict_keys(['uncorrelated_gaussian_centered_sigma_0.1_histogram_1.pkl', 'uncorrelated_gaussian_centered_sigma_0.3_KDE_1.pkl', 'uncorrelated_gaussian_centered_sigma_0.1_KDE_0.pkl', 'uncorrelated_gaussian_centered_sigma_0.1_KDE_1.pkl', 'uncorrelated_gaussian_centered_sigma_0.8_KDE_1.pkl', 'uncorrelated_gaussian_centered_sigma_0.5_histogram_1.pkl', 'uncorrelated_gaussian_centered_sigma_0.8_histogram_1.pkl', 'uncorrelated_gaussian_centered_sigma_0.1_histogram_0.pkl', 'uncorrelated_gaussian_centered_sigma_0.8_histogram_0.pkl', 'uncorrelated_gaussian_centered_sigma_0.3_histogram_1.pkl', 'uncorrelated_gaussian_centered_sigma_0.5_KDE_0.pkl', 'uncorrelated_gaussian_centered_sigma_0.5_KDE_1.pkl', 'uncorrelated_gaussian_centered_sigma_0.3_histogram_0.pkl', 'uncorrelated_gaussian_centered_sigma_0.8_KDE_0.pkl', 'uncorrelated_gaussian_centered_sigma_0.5_histogram_0.pkl', 'uncorrelated_gaussian_centered_sigma_0.3_KDE_0.pkl'])
dict_keys([])


### Analyze Multi Digit series

In [5]:
import importlib
import baseline_models
importlib.reload(baseline_models)

from baseline_models import *

In [6]:
for series_name, series_dict in sorted(continuous_series_task.items()):   
    llama_size = series_dict['llama_size']
    prec = series_dict['prec']
    rescaled_full_series = series_dict['rescaled_full_series']
    if llama_size == 'KDE':
        print("Processing ", series_name)
        PDF_list = KDE_for_series(rescaled_full_series,'gaussian',prec)
        series_dict['PDF_list'] = PDF_list
        save_name = os.path.join(save_path, series_name)
        with open(save_name, 'wb') as f:
            pickle.dump(series_dict, f)
            
    if llama_size == 'histogram':
        print("Processing ", series_name)
        PDF_list = histogram_for_series(rescaled_full_series,prec)
        series_dict['PDF_list'] = PDF_list
        save_name = os.path.join(save_path, series_name)
        with open(save_name, 'wb') as f:
            pickle.dump(series_dict, f)            


Processing  uncorrelated_gaussian_centered_sigma_0.1_KDE_0.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.1_KDE_1.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.1_histogram_0.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.1_histogram_1.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.3_KDE_0.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.3_KDE_1.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.3_histogram_0.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.3_histogram_1.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.5_KDE_0.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.5_KDE_1.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.5_histogram_0.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.5_histogram_1.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.8_KDE_0.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.8_KDE_1.pkl
Processing  uncorrelated_gaussian_centered_sigma_0.8_histogram_0.pkl
Processing  u

In [8]:
PDF_list[0].bin_center_arr

array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05,
       1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95, 2.05, 2.15,
       2.25, 2.35, 2.45, 2.55, 2.65, 2.75, 2.85, 2.95, 3.05, 3.15, 3.25,
       3.35, 3.45, 3.55, 3.65, 3.75, 3.85, 3.95, 4.05, 4.15, 4.25, 4.35,
       4.45, 4.55, 4.65, 4.75, 4.85, 4.95, 5.05, 5.15, 5.25, 5.35, 5.45,
       5.55, 5.65, 5.75, 5.85, 5.95, 6.05, 6.15, 6.25, 6.35, 6.45, 6.55,
       6.65, 6.75, 6.85, 6.95, 7.05, 7.15, 7.25, 7.35, 7.45, 7.55, 7.65,
       7.75, 7.85, 7.95, 8.05, 8.15, 8.25, 8.35, 8.45, 8.55, 8.65, 8.75,
       8.85, 8.95, 9.05, 9.15, 9.25, 9.35, 9.45, 9.55, 9.65, 9.75, 9.85,
       9.95])